In [ ]:
import pandas as pd
import pathlib
import networkx
import matplotlib.pyplot as plt

from clustering.ClusterSets.HierarchicalClusterSets import HierarchicalClusterSets
from clustering.ClusterSets.KMeansClusterSets import KMeansClusterSets
from clustering.Snapshots import Snapshots
from utils import paths
from networks.TemporalNetwork import TemporalNetwork

## Declare parameters to use throughout

In [ ]:
distance_metric = 'euclidean'
cluster_method = 'ward'
cluster_limit_type = 'maxclust'

cluster_limit_range = [2 + (1 * i) for i in range(0, 10)]
events = [(24 + 12 * i, '', 'dashed') for i in range(4)]
phases = [
    (18, 24, 'N'),
    (24, 36, 'Day'),
    (36, 48, 'Night'),
    (48, 60, 'Day'),
    (60, 66, 'N')]
time_ticks = [18 + 6*i for i in range(8 + 1)]


output_directory = '../data/output'

### Load/create temporal networks

In [ ]:
node_table_filepath = '../data/temporal_data/circadian/circadian_temporal_node_data_24_hour_genes_mean_normalised_full.csv'
static_network_filepath = '../data/static_networks/circadian_full.edgelist'
binary = True
normalise = None
thresholds = [round(0.1 + (0.1 * i), 1) for i in range(3)]

node_table = pd.read_csv(node_table_filepath, sep='\t', index_col=0)
static_network = networkx.read_edgelist(static_network_filepath, delimiter=', ')

temporal_networks = [
    TemporalNetwork.from_static_network_and_node_table_dataframe(
        static_network,
        node_table,
        combine_node_weights=lambda x, y: x*y,
        threshold=threshold,
        binary=binary,
        normalise=normalise)
    for threshold
    in thresholds
]

### Calculate cluster sets for each threshold

In [ ]:
valid_cluster_sets = []
for threshold, temporal_network in zip(thresholds, temporal_networks):
    try:
        snapshots = Snapshots.from_temporal_network(temporal_network, cluster_method, distance_metric)
        constructor = HierarchicalClusterSets if cluster_method != 'k_means' else KMeansClusterSets
        cluster_sets = constructor(snapshots, cluster_limit_type, cluster_limit_range)
        valid_cluster_sets.append((cluster_sets, threshold))
    except Exception as e:
        print(f'Error when threshold = {threshold}: {e}')

### Plot across different thresholds

In [ ]:
gridspec_kw = {"width_ratios": [3, 1, 2]}
figsize = (9, 4*len(valid_cluster_sets))
fig, axs = plt.subplots(len(valid_cluster_sets), 3, figsize=figsize, gridspec_kw=gridspec_kw, sharey='row', sharex='col')

for i, (cluster_sets, threshold) in enumerate(valid_cluster_sets):
    row = (axs[i, 0], axs[i, 1], axs[i, 2])
    cluster_sets.plot_and_format_with_average_silhouettes(row, events, phases, time_ticks)

    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    fontdict = {'horizontalalignment': 'left'}
    axs[i, 0].set_title(f'Clusters and silhouette scores (threshold = {threshold})', fontdict=fontdict, pad=30)

title = f'{pathlib.Path(node_table_filepath).stem}, binary={binary}, normalise={normalise}'
title += f'\nmetric={distance_metric}, cluster_method={cluster_method}, cluster_limit_type={cluster_limit_type}'
fig.suptitle(title, y=0.98, weight='bold')

# Save
if output_directory is not None:
    filename = f"{output_directory}/silhouette_scores_over_thresholds_{paths.slugify(title)}"
    plt.savefig(f"{filename}.png", dpi=250, bbox_inches="tight")